<a href="https://colab.research.google.com/github/sneha2180/HOUSE-PRICE-PREDICTION/blob/main/boston_housing_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
# Reading the file using pandas
df = pd.read_csv('housing.csv')

In [9]:
# Performing EDA (Exploratory Data Analysis) on the DataSets.

In [10]:
# head method to check aboce 5 entries of the datasets
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [11]:
df.info

<bound method DataFrame.info of         price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no

In [12]:
# check the column name
df.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [13]:
# checking the dimensions of the dataset
df.shape

(545, 13)

In [14]:
# checking the datatypes of the variable
df.dtypes

price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object

In [15]:
# To check individual column type
type(df['price'])

pandas.core.series.Series

In [16]:
# To check the individual column entries
df[['price']]

,price
0,13300000
1,12250000
2,12250000
3,12215000
4,11410000
...,...
540,1820000
541,1767150
542,1750000
543,1750000


In [17]:
# Check column unique value counts
df['furnishingstatus'].value_counts()

semi-furnished    227
unfurnished       178
furnished         140
Name: furnishingstatus, dtype: int64

In [18]:
# As the machine learning model doesn't understand the text so we need to change the text to int
# First step to append the objects to a certain list

cat_col_list = list(df.select_dtypes('object').columns)
cat_col_list

['mainroad',
 'guestroom',
 'basement',
 'hotwaterheating',
 'airconditioning',
 'prefarea',
 'furnishingstatus']

In [19]:
# Direct method to append the complete columns to a variable

df_cat = df.select_dtypes('object')
df_cat

,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus
0,yes,no,no,no,yes,yes,furnished
1,yes,no,no,no,yes,no,furnished
2,yes,no,yes,no,no,yes,semi-furnished
3,yes,no,yes,no,yes,yes,furnished
4,yes,yes,yes,no,yes,no,furnished
...,...,...,...,...,...,...,...
540,yes,no,yes,no,no,no,unfurnished
541,no,no,no,no,no,no,semi-furnished
542,yes,no,no,no,no,no,unfurnished
543,no,no,no,no,no,no,furnished


In [23]:
# To change the text to numerical value there are certain methods
# Import following classes
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [24]:
# A complete process to change the text values to numerical values
# Label Binarizer can only change 2 values to 0 or 1
# label Encode can change more then 2 uniqu values to intergers.

label_binarizer_cols = []
label_encoder_col = []

In [25]:
# Assigning the columns to respective lists

for each_col in df_cat.columns:
    if len(df_cat[each_col].unique()) == 2:
        label_binarizer_cols.append(each_col)
    else:
        label_encoder_col.append(each_col)
    print(df_cat[each_col].unique())

['yes' 'no']
['no' 'yes']
['no' 'yes']
['no' 'yes']
['yes' 'no']
['yes' 'no']
['furnished' 'semi-furnished' 'unfurnished']


In [26]:
# Conversion of Binarizer values and updation of the datasets

lb_obj = {}
for each_col in label_binarizer_cols:
    lb = LabelBinarizer()
    lb_obj[each_col] = lb
    df[each_col] = lb.fit_transform(df[each_col])

In [27]:
# Data Set View 
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,unfurnished
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,semi-furnished
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,unfurnished
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,furnished


In [28]:
# Conversion of Encoder values and updation of the datasets

le_obj = {}
for each_col in label_encoder_col:
    le = LabelEncoder()
    le_obj[each_col] = le
    df[each_col] = le.fit_transform(df[each_col])

In [29]:
# Data Base View

df[['furnishingstatus']]

,furnishingstatus
0,0
1,0
2,1
3,0
4,0
...,...
540,2
541,1
542,2
543,0


In [30]:
# Assigning of the necessary variable 

x = df.drop('price', axis = 1)
y = df['price']

In [31]:
x.head(5)

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,7420,4,2,3,1,0,0,0,1,2,1,0
1,8960,4,4,4,1,0,0,0,1,3,0,0
2,9960,3,2,2,1,0,1,0,0,2,1,1
3,7500,4,2,2,1,0,1,0,1,3,1,0
4,7420,4,1,2,1,1,1,0,1,2,0,0


In [32]:
y.head()

0    13300000
1    12250000
2    12250000
3    12215000
4    11410000
Name: price, dtype: int64

In [33]:
# Model Building using 3 different algorithms

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV

In [34]:
# Linear Regression model
# Assigning the algorithm to the variable
lr = LinearRegression()

# Fitting of the model
lr.fit(x,y)


LinearRegression()

In [35]:
# Prediction of the first 50 entries
lr.predict(x)[1:50]

array([10638380.23666685,  7542628.07268391,  8408698.53020236,
        6771486.96259187,  8334747.49458948,  9710889.51208249,
        8514703.09449342,  7584399.24465679,  7717327.43527944,
        8410819.47648123,  8209030.40806563,  7046301.57103505,
        6167605.81723847,  6098291.56137894,  5060287.29954374,
        7501266.00075274,  8149146.47006861,  6595157.0812944 ,
        6895119.28922269,  5403985.36960561,  6699498.45778087,
        6075496.76011549,  6663541.52845016,  7321535.50104423,
        8192792.78781793,  8069632.6612009 ,  4553941.69895367,
        7304016.64916945,  7148125.69943508,  7472766.34770253,
        6581210.07040689,  6852164.8507537 ,  6713890.30012285,
        6469709.71705743,  7783286.13916525,  7837030.13481458,
        8390574.30017046,  6422145.90845984,  7165880.21952014,
        6229267.40137125,  7714670.05526346,  7230078.99486997,
        6592032.26201704,  7379068.00097786,  6316228.67806435,
        7259593.61451047,  7623676.33306

In [36]:
# RidgeCV model
# Assigning the algorithm to the variable
rc = RidgeCV()

# Fitting of the model
rc.fit(x,y)


RidgeCV()

In [37]:
# Prediction of the first 50 entries
rc.predict(x)[1:50]

array([10627867.39543733,  7543095.9467725 ,  8400911.08251486,
        6775255.63429963,  8315012.35328818,  9698176.52684408,
        8521929.48215834,  7582154.86507871,  7703934.92986606,
        8412990.53372998,  8167823.06238235,  7034718.7604987 ,
        6132858.22352633,  6094766.18229555,  5072044.91741868,
        7487616.4401832 ,  8149229.83705756,  6587588.06592665,
        6881681.06663874,  5374076.51005018,  6690984.49938719,
        6075855.23021056,  6654858.3562301 ,  7319194.75837424,
        8182096.44468209,  8056446.42343246,  4564675.01209159,
        7273329.96122913,  7135366.58282994,  7469758.43230029,
        6585621.15508806,  6839918.53334929,  6707173.71495503,
        6473654.94903523,  7781851.19133382,  7798291.68562695,
        8392887.5968627 ,  6423540.90842809,  7163169.82017708,
        6222786.89739116,  7703965.5735791 ,  7226063.93063687,
        6599758.22206495,  7377384.48526677,  6309470.38488403,
        7256186.27403621,  7622968.24618

In [38]:
# LassoCV model
# Assigning the algorithm to the variable
lc = LassoCV()

# Fitting of the model
lc.fit(x,y)

LassoCV()

In [39]:
# Prediction of the first 50 entries
lc.predict(x)[1:50]

array([6524843.66079319, 6986356.58017171, 5851034.79850057,
       5814113.76495029, 5851034.79850057, 6349468.75142936,
       9866197.19709361, 6127942.55012768, 5043387.18958818,
       8481658.43895808, 5158765.4194328 , 5412597.52509099,
       4004983.12098653, 5989488.67431412, 5158765.4194328 ,
       5435673.17105991, 6312547.71787908, 4512647.33230289,
       5352600.84557178, 4383423.71487691, 5691812.84131498,
       6104866.90415875, 4494186.81552775, 6451001.59369263,
       5407982.3958972 , 5158765.4194328 , 6485615.06264602,
       6058715.6122209 , 4928008.95974355, 5839496.97551611,
       5620278.33881131, 4641870.94972887, 5140304.90265766,
       5546436.27171075, 5620278.33881131, 5842727.56595176,
       6543304.17756834, 5158765.4194328 , 5158765.4194328 ,
       5412597.52509099, 5324910.07040907, 5380291.62073449,
       5158765.4194328 , 5158765.4194328 , 5158765.4194328 ,
       5158765.4194328 , 5435673.17105991, 4374193.45648934,
       5823344.02333786]

In [40]:
# Mean absolute error to check for all the 3 algorithms
from sklearn.metrics import mean_absolute_error

In [41]:
# MAE for Linear Regression Model
mean_absolute_error(y, lr.predict(x))

776190.8030888345

In [42]:
# MAE for RidgeCV Model
mean_absolute_error(y, rc.predict(x))

775398.6923268461

In [ ]:
# MAE for LassoCV Model
mean_absolute_error(y, lc.predict(x))

1172348.5953106831